In [1]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import json

In [2]:
#Define Executable Path for ChromeDriver to use for web scraping

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Define base endpoint URL. Other url's will be built off this endpoint.

espn_url = 'https://www.espn.com'

In [4]:
# Define UNC extension and URL

tarheel_ext = '/mens-college-basketball/team/_/id/153/north-carolina-tar-heels'
tarheel_team_page = espn_url + tarheel_ext

browser.visit(tarheel_team_page)

In [5]:
#Use Beautiful Soup HTML.parser to extract relvant details about news articles

html = browser.html
soup = bs(html, 'html.parser')

In [6]:
# Extract list of games

game_listings = soup.find_all('section', class_="club-schedule")[0].find_all('li')
game_listings

[<li><a class="" href="/mens-college-basketball/game/_/gameId/401310911" name="&amp;lpos=mens-college-basketball:teamclubhouse:schedule:post" rel=""><div class="logo"><img data-behavior="fix_broken_images" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/ncaa/500/275.png&amp;h=40&amp;w=40"/></div><div class="game-info">vs <span class="rank">9</span> WISC</div><div class="game-meta"><div class="game-result loss">L</div><div class="score">85-62</div></div></a></li>,
 <li><a class="" href="/mens-college-basketball/game/_/gameId/401252794" name="&amp;lpos=mens-college-basketball:teamclubhouse:schedule:regular" rel=""><div class="logo"><img data-behavior="fix_broken_images" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/ncaa/500/232.png&amp;h=40&amp;w=40"/></div><div class="game-info">vs  COFC</div><div class="game-meta"><div class="game-result win">W</div><div class="score">79-60</div></div></a></li>,
 <li><a class="" href="/mens-college-basketball/game/_/gameId/401255064" na

In [7]:
game_urls = []
for game in game_listings:
    new_url = game.a['href']
    game_url = espn_url + new_url
    game_urls.append(game_url)

In [8]:
game_urls

['https://www.espn.com/mens-college-basketball/game/_/gameId/401310911',
 'https://www.espn.com/mens-college-basketball/game/_/gameId/401252794',
 'https://www.espn.com/mens-college-basketball/game/_/gameId/401255064',
 'https://www.espn.com/mens-college-basketball/game/_/gameId/401256578',
 'https://www.espn.com/mens-college-basketball/game/_/gameId/401256579',
 'https://www.espn.com/mens-college-basketball/game/_/gameId/401256095',
 'https://www.espn.com/mens-college-basketball/game/_/gameId/401268610',
 'https://www.espn.com/mens-college-basketball/game/_/gameId/401270409',
 'https://www.espn.com/mens-college-basketball/game/_/gameId/401260181',
 'https://www.espn.com/mens-college-basketball/game/_/gameId/401265053',
 'https://www.espn.com/mens-college-basketball/game/_/gameId/401274413',
 'https://www.espn.com/mens-college-basketball/game/_/gameId/401265057',
 'https://www.espn.com/mens-college-basketball/game/_/gameId/401260205',
 'https://www.espn.com/mens-college-basketball/game

In [9]:
opp_id = 0
game_id = []
opponent = []
home_away_game = []
game_dfs = []
for game in game_urls:
    try:
        browser.visit(game)
        temp_html = browser.html
        temp_soup = bs(temp_html, 'html.parser')
        boxxy_score_endpoint = temp_soup.find('li', class_="sub boxscore").a['href']
        another_temp = espn_url + boxxy_score_endpoint
        browser.visit(another_temp)
        another_html = browser.html
        another_soup = bs(another_html, 'html.parser')
        teams = another_soup.find('div',class_="row-wrapper")
        away_team = teams.find('div',class_="col column-one gamepackage-away-wrap").find('div',class_='team-name').text
        home_team = teams.find('div',class_="col column-two gamepackage-home-wrap").find('div',class_='team-name').text
        print(f'The Away Team is {away_team}')
        print(f'The Home Team is {home_team}')
        tables = pd.read_html(another_temp)
        opp_id += 1
        if away_team == "North Carolina":
            #opp_id = opp_id =+ 1
            game_id.append(opp_id)
            opponent.append(home_team)
            home_away_game.append('Away')
            print(opp_id)
            heels = tables[1]
        else:
            #opp_id = opp_id =+ 1
            game_id.append(opp_id)
            opponent.append(away_team)
            home_away_game.append('Home')
            print(opp_id)
            heels = tables[2]
        heels.drop(heels.tail(2).index,inplace=True)
        players = heels['Starters']['Bench']
        minutes = heels['MIN']['MIN']
        field_goals = heels['FG']['FG']
        free_throws = heels['FT']['FT']
        three_pointers = heels['3PT']['3PT']
        off_reb = heels['OREB']['OREB']
        def_reb = heels['DREB']['DREB']
        total_reb = heels['REB']['REB']
        assists = heels['AST']['AST']
        steals = heels['STL']['STL']
        blocks = heels['BLK']['BLK']
        turnovers = heels['TO']['TO']
        points = heels['PTS']['PTS']
        players_names = []
        for player in players:
            try:
                #print(player)
                length = len(player)
                splitter = (length-1)/2
                splitter_int = int(splitter)
                player_name = player[:splitter_int]
                players_names.append(player_name)
            except TypeError:
                print('Doesnt matter!')
        game_df = pd.DataFrame({
        "Player": players_names,
        "MP": minutes,
        "FG": field_goals,
        "FT": free_throws,
        "3P": three_pointers,
        "OREB": off_reb,
        "DREB": def_reb,
        "TOT REB": total_reb,
        "AST": assists,
        "STL": steals,
        "BLK": blocks,
        "TO": turnovers,
        "PTS": points
        })
        game_df['MP'] = game_df['MP'].astype(int)
        game_df['FGA'] = game_df['FG'].str.split(pat="-")[0][0]
        game_df['FGM'] = game_df['FG'].str.split(pat="-")[0][1]
        game_df['FTA'] = game_df['FT'].str.split(pat="-")[0][0]
        game_df['FTM'] = game_df['FT'].str.split(pat="-")[0][1]
        game_df['3PA'] = game_df['3P'].str.split(pat="-")[0][0]
        game_df['3PM'] = game_df['3P'].str.split(pat="-")[0][1]
        game_df['OREB'] = game_df['OREB'].astype(int)
        game_df['DREB'] = game_df['DREB'].astype(int)
        game_df['TOT REB'] = game_df['TOT REB'].astype(int)
        game_df['AST'] = game_df['AST'].astype(int)
        game_df['STL'] = game_df['STL'].astype(int)
        game_df['BLK'] = game_df['BLK'].astype(int)
        game_df['TO'] = game_df['TO'].astype(int)
        game_df['PTS'] = game_df['PTS'].astype(int)
        game_df['GAME_ID'] = opp_id
        cleaned_game_df = game_df.drop(['FG','FT','3P'],axis=1)
        cleaned_game_df = cleaned_game_df[['GAME_ID','Player','MP','FGA','FGM','3PA','3PM','FTA','FTM','OREB','DREB','TOT REB','AST','STL', \
                                      'BLK', 'TO', 'PTS']]
        game_dfs.append(cleaned_game_df)
    except AttributeError:
        print('Game was cancelled or postponed')

The Away Team is Wisconsin
The Home Team is North Carolina
1
The Away Team is Charleston
The Home Team is North Carolina
2
The Away Team is North Carolina
The Home Team is UNLV
3
The Away Team is North Carolina
The Home Team is Stanford
4
The Away Team is Texas
The Home Team is North Carolina
5
The Away Team is North Carolina
The Home Team is Iowa
6
The Away Team is North Carolina Central
The Home Team is North Carolina
7
The Away Team is North Carolina
The Home Team is Kentucky
8
The Away Team is North Carolina
The Home Team is NC State
9
The Away Team is North Carolina
The Home Team is Georgia Tech
10
The Away Team is Notre Dame
The Home Team is North Carolina
11
The Away Team is North Carolina
The Home Team is Miami
12
Game was cancelled or postponed
The Away Team is Syracuse
The Home Team is North Carolina
13
The Away Team is North Carolina
The Home Team is Florida State
14
The Away Team is Wake Forest
The Home Team is North Carolina
15
The Away Team is NC State
The Home Team is No

In [10]:
print(game_id)
print(opponent)
print(home_away_game)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
['Wisconsin', 'Charleston', 'UNLV', 'Stanford', 'Texas', 'Iowa', 'North Carolina Central', 'Kentucky', 'NC State', 'Georgia Tech', 'Notre Dame', 'Miami', 'Syracuse', 'Florida State', 'Wake Forest', 'NC State', 'Pittsburgh', 'Clemson', 'Duke', 'Virginia', 'Northeastern', 'Louisville', 'Marquette', 'Florida State', 'Syracuse', 'Duke', 'Notre Dame', 'Virginia Tech', 'Florida State']
['Home', 'Home', 'Away', 'Away', 'Home', 'Away', 'Home', 'Away', 'Away', 'Away', 'Home', 'Away', 'Home', 'Away', 'Home', 'Home', 'Away', 'Away', 'Away', 'Away', 'Home', 'Home', 'Home', 'Home', 'Away', 'Home', 'Home', 'Away', 'Away']


In [11]:
game_dfs[0]

,GAME_ID,Player,MP,FGA,FGM,3PA,3PM,FTA,FTM,OREB,DREB,TOT REB,AST,STL,BLK,TO,PTS
0,1,A. Bacot,24,6,9,0,0,3,3,2,2,4,1,2,0,1,15
1,1,G. Brooks,22,6,9,0,0,3,3,5,5,10,1,0,2,2,10
2,1,K. Walton,26,6,9,0,0,3,3,0,1,1,1,0,0,0,8
3,1,C. Love,28,6,9,0,0,3,3,0,1,1,3,0,0,0,10
4,1,L. Black,22,6,9,0,0,3,3,0,3,3,0,0,0,1,2
5,1,D. Farris,1,6,9,0,0,3,3,0,0,0,0,0,0,0,0
6,1,W. Kessler,13,6,9,0,0,3,3,0,1,1,1,0,0,1,2
7,1,D. Sharpe,19,6,9,0,0,3,3,1,2,3,1,0,1,3,4
8,1,R. McAdoo,1,6,9,0,0,3,3,0,0,0,0,0,0,0,0
9,1,W. Miller,1,6,9,0,0,3,3,0,0,0,0,0,0,1,0


In [12]:
game_dfs[1]

,GAME_ID,Player,MP,FGA,FGM,3PA,3PM,FTA,FTM,OREB,DREB,TOT REB,AST,STL,BLK,TO,PTS
0,2,A. Bacot,22,4,5,0,0,3,4,3,5,8,0,0,3,1,11
1,2,G. Brooks,28,4,5,0,0,3,4,2,9,11,1,0,2,4,7
2,2,R.J. Davis,27,4,5,0,0,3,4,2,2,4,1,1,0,1,11
3,2,C. Love,26,4,5,0,0,3,4,0,1,1,4,2,0,1,17
4,2,L. Black,28,4,5,0,0,3,4,1,1,2,1,1,1,1,4
5,2,D. Farris,1,4,5,0,0,3,4,0,0,0,0,0,0,0,0
6,2,W. Kessler,5,4,5,0,0,3,4,2,2,4,0,0,0,1,6
7,2,D. Sharpe,22,4,5,0,0,3,4,5,5,10,2,2,1,0,13
8,2,R. McAdoo,1,4,5,0,0,3,4,0,0,0,0,0,0,0,0
9,2,W. Miller,2,4,5,0,0,3,4,0,0,0,0,0,0,0,2


In [13]:
game_dfs[10]

,GAME_ID,Player,MP,FGA,FGM,3PA,3PM,FTA,FTM,OREB,DREB,TOT REB,AST,STL,BLK,TO,PTS
0,11,D. Sharpe,29,9,19,0,0,7,9,8,2,10,1,1,0,2,25
1,11,A. Bacot,17,9,19,0,0,7,9,5,5,10,1,1,0,1,4
2,11,K. Walton,27,9,19,0,0,7,9,0,1,1,3,0,0,1,12
3,11,C. Love,15,9,19,0,0,7,9,0,0,0,0,0,0,2,5
4,11,L. Black,31,9,19,0,0,7,9,3,1,4,3,0,0,0,8
5,11,W. Kessler,5,9,19,0,0,7,9,1,1,2,0,0,1,1,2
6,11,G. Brooks,29,9,19,0,0,7,9,3,5,8,4,1,0,0,5
7,11,P. Johnson,9,9,19,0,0,7,9,1,1,2,0,1,1,0,2
8,11,R.J. Davis,23,9,19,0,0,7,9,0,3,3,2,0,0,2,3
9,11,A. Platek,15,9,19,0,0,7,9,0,5,5,1,0,0,0,0


In [ ]:
home_team_2 = teams.find('div',class_="col column-two gamepackage-home-wrap").find_all('td', class_="name")
players_full_names = []
for player in home_team_2:
    try:
        player_url = player.a['href']
        browser.visit(player_url)
        player_html = browser.html
        player_soup = bs(player_html, 'html.parser')
        player_info = player_soup.find('div', class_='PlayerHeader__Main flex items-center')
        player_image = player_info.find('div',class_='Image__Wrapper Image__Wrapper--relative').img['src']
        player_name_bin = player_info.find('h1',class_='PlayerHeader__Name flex flex-column ttu fw-bold pr4 h3').find_all('span')
        first_name = player_name_bin[0].text
        last_name = player_name_bin[1].text
        players_full_names.append(f'{first_name} {last_name}')
    except TypeError:
        print('End of Dataset')
        
players_full_names

In [14]:
cumulative_game_stats = game_dfs[0]

In [15]:
season_game_stats = cumulative_game_stats.append(game_dfs[1:29])

In [18]:
season_game_stats.loc[season_game_stats['GAME_ID'] == 29]

,GAME_ID,Player,MP,FGA,FGM,3PA,3PM,FTA,FTM,OREB,DREB,TOT REB,AST,STL,BLK,TO,PTS
0,29,A. Bacot,26,5,10,0,0,2,3,4,2,6,1,2,2,0,12
1,29,G. Brooks,27,5,10,0,0,2,3,5,3,8,1,0,0,1,5
2,29,K. Walton,28,5,10,0,0,2,3,0,5,5,1,2,0,2,11
3,29,C. Love,26,5,10,0,0,2,3,1,1,2,1,2,0,4,13
4,29,L. Black,24,5,10,0,0,2,3,3,1,4,1,1,1,1,7
5,29,W. Kessler,10,5,10,0,0,2,3,1,2,3,0,0,0,0,2
6,29,D. Sharpe,17,5,10,0,0,2,3,2,4,6,0,0,1,1,5
7,29,A. Harris,14,5,10,0,0,2,3,1,2,3,2,0,0,0,4
8,29,R.J. Davis,24,5,10,0,0,2,3,0,1,1,2,1,0,2,7
9,29,A. Platek,4,5,10,0,0,2,3,0,0,0,0,0,0,0,0


In [19]:
season_game_stats

,GAME_ID,Player,MP,FGA,FGM,3PA,3PM,FTA,FTM,OREB,DREB,TOT REB,AST,STL,BLK,TO,PTS
0,1,A. Bacot,24,6,9,0,0,3,3,2,2,4,1,2,0,1,15
1,1,G. Brooks,22,6,9,0,0,3,3,5,5,10,1,0,2,2,10
2,1,K. Walton,26,6,9,0,0,3,3,0,1,1,1,0,0,0,8
3,1,C. Love,28,6,9,0,0,3,3,0,1,1,3,0,0,0,10
4,1,L. Black,22,6,9,0,0,3,3,0,3,3,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,29,W. Kessler,10,5,10,0,0,2,3,1,2,3,0,0,0,0,2
6,29,D. Sharpe,17,5,10,0,0,2,3,2,4,6,0,0,1,1,5
7,29,A. Harris,14,5,10,0,0,2,3,1,2,3,2,0,0,0,4
8,29,R.J. Davis,24,5,10,0,0,2,3,0,1,1,2,1,0,2,7


In [20]:
season_game_stats.to_csv('game_stats.csv', index=False)

In [21]:
opponent_df = pd.DataFrame({
        "GAME_ID": game_id,
        "OPPONENT": opponent,
        "HEELS_HOME/AWAY": home_away_game,
        })

In [23]:
opponent_df.to_csv('opponents.csv', index=False)